In [6]:
# Setup the Jupyter version of Dash
from jupyter_dash import JupyterDash

# Configure the necessary Python module imports
import base64
import dash_leaflet as dl
from dash import dcc, html
import plotly.express as px
from dash import dash_table
from dash.dependencies import Input, Output
JupyterDash.infer_jupyter_proxy_config()

# Configure the plotting routines
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


#### FIX ME #####
# change animal_shelter and AnimalShelter to match your CRUD Python module file name and class name
from CRUD_Python_Module import AnimalShelter



###########################
# Data Manipulation / Model
###########################
# FIX ME update with your username and password and CRUD Python module name. NOTE: You will
# likely need more variables for your constructor to handle the hostname and port of the MongoDB
# server, and the database and collection names

username = "aacuser"
password = "Password123!"
shelter = AnimalShelter(username, password)


# class read method must support return of list object and accept projection json input
# sending the read method an empty document requests all documents be returned
df = pd.DataFrame.from_records(shelter.read({}))

# MongoDB v5+ is going to return the '_id' column and that is going to have an 
# invlaid object type of 'ObjectID' - which will cause the data_table to crash - so we remove
# it in the dataframe here. The df.drop command allows us to drop the column. If we do not set
# inplace=True - it will reeturn a new dataframe that does not contain the dropped column(s)
df.drop(columns=['_id'],inplace=True)

## Debug
# print(len(df.to_dict(orient='records')))
# print(df.columns)


#########################
# Dashboard Layout / View
#########################
app = JupyterDash('SimpleExample')

# Load Grazioso Salvare logo
image_filename = 'Grazioso Salvare Logo.png'

# Read + encode the image
encoded_image = base64.b64encode(open(image_filename, 'rb').read()).decode()

# Build final logo string
logo_src = 'data:image/png;base64,{}'.format(encoded_image)

app.layout = html.Div([
    html.Div(id='hidden-div', style={'display':'none'}),

    html.Center(html.B(html.H1('SNHU CS-340 Dashboard'))),

    html.Center(
        html.Img(
            src=logo_src,
            style={'width':'250px'}
        )
    ),

    html.Center(html.H4("Dashboard by Jael Ortiz")),

    html.Hr(),
    html.Div([
    html.Label("Filter by Rescue Type"),
    dcc.RadioItems(
        id='filter-type',
        options=[
            {'label': 'Show All', 'value': 'reset'},
            {'label': 'Water Rescue', 'value': 'water'},
            {'label': 'Mountain/Wilderness Rescue', 'value': 'mountain'},
            {'label': 'Disaster/Tracking', 'value': 'disaster'},
        ],
        value='reset', 
        inline=True
    )
]),
    dash_table.DataTable(
        id='datatable-id',
        columns=[
            {"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns
        ],
        data=df.to_dict('records'),
        #FIXME: Set up the features for your interactive data table to make it user-friendly for your client
        page_size=10,
        sort_action="native",
        filter_action="native",
        row_selectable="single",
        selected_rows=[0],
        style_table={'height': '400px', 'overflowY': 'auto'},

    ),
    html.Br(),
    html.Hr(),
    html.Div(
            id='map-id',
            className='col s12 m6',
            ),
    
    html.Br(),
    html.Div(
            id='chart-id',
            className='col s12 m6',
)
])

#############################################
# Interaction Between Components / Controller
#############################################

@app.callback(
    Output('datatable-id', 'data'),
    [Input('filter-type', 'value')]
)
def update_dashboard(filter_type):
    # Show ALL records (reset)
    if filter_type == 'reset':
        df_filtered = shelter.read({})
    
    # Water Rescue filter
    elif filter_type == 'water':
        df_filtered = shelter.read({
            "animal_type": "Dog",
            "breed": {
                "$in": [
                    "Labrador Retriever Mix", "Chesapeake Bay Retriever",
                    "Newfoundland"
                ]
            }
        })

    # Mountain / Wilderness Rescue
    elif filter_type == 'mountain':
        df_filtered = shelter.read({
            "animal_type": "Dog",
            "breed": {
                "$in": [
                    "German Shepherd", "Alaskan Malamute",
                    "Old English Sheepdog", "Siberian Husky",
                    "Rottweiler"
                ]
            }
        })

    # Disaster / Tracking
    elif filter_type == 'disaster':
        df_filtered = shelter.read({
            "animal_type": "Dog",
            "breed": {
                "$in": [
                    "Doberman Pinscher", "German Shepherd",
                    "Golden Retriever", "Bloodhound",
                    "Rottweiler"
                ]
            }
        })

    else:
        df_filtered = shelter.read({})

    # Convert to DataFrame → Drop _id column → Return dict for table
    df_f = pd.DataFrame.from_records(df_filtered)
    if '_id' in df_f.columns:
        df_f = df_f.drop(columns=['_id'])

    return df_f.to_dict('records')

#This callback will highlight a row on the data table when the user selects it
@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    if not selected_columns:
        return []

    return [{
        'if': {'column_id': i},
        'background_color': '#D2F3FF'
    } for i in selected_columns]


# This callback will update the geo-location chart for the selected data entry
# derived_virtual_data will be the set of data available from the datatable in the form of 
# a dictionary.
# derived_virtual_selected_rows will be the selected row(s) in the table in the form of
# a list. For this application, we are only permitting single row selection so there is only
# one value in the list.
# The iloc method allows for a row, column notation to pull data from the datatable
#FIXME Add in the code for your geolocation chart

@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_virtual_data"),
     Input('datatable-id', "derived_virtual_selected_rows")]
)
def update_map(viewData, index):
    if viewData is None or len(viewData) == 0:
        return []

    # Convert table into DataFrame
    dff = pd.DataFrame.from_dict(viewData)

    if index is None or len(index) == 0:
        row = 0
    else:
        row = index[0]
    # Build the map
    return [
        dl.Map(
            style={'width': '1000px', 'height': '500px'},
            center=[30.75, -97.48], 
            zoom=10, 
            children=[
                dl.TileLayer(id="base-layer-id"),

                # Marker using coordinates in columns 13 & 14
                dl.Marker(
                    position=[dff.iloc[row, 13], dff.iloc[row, 14]],
                    children=[
                        dl.Tooltip(dff.iloc[row, 4]),  # breed column
                        dl.Popup([
                            html.H1("Animal Name"),
                            html.P(dff.iloc[row, 9])      # name column
                        ])
                    ]
                )
            ]
        )
    ]

@app.callback(
    Output('chart-id', 'children'),
    [Input('datatable-id', 'derived_virtual_data')]
)
def update_chart(viewData):
    if not viewData:
        return [html.P("No data available")]

    dff = pd.DataFrame.from_dict(viewData)

    # Automatically detect the breed column
    breed_col = None
    for col in dff.columns:
        if "breed" in col.lower():
            breed_col = col
            break

    # If no breed column exists, return empty
    if breed_col is None:
        return [html.P("Breed column not found")]

    # Clean and count values
    dff[breed_col] = dff[breed_col].fillna("Unknown").astype(str)
    counts = dff[breed_col].value_counts().reset_index()
    counts.columns = ["Breed", "Count"]

    # Build chart
    fig = px.bar(
        counts,
        x="Breed",
        y="Count",
        title="Breed Count in Filtered Results"
    )

    return [dcc.Graph(figure=fig)]




    
# Run app and display result in jupyterlab mode, note, if you have previously run a prior app, the default port of 8050 may not be available, if so, try setting an alternate port.
app.run_server()

Dash app running on https://avatarbritish-palacemother-3000.codio.io/proxy/8050/
